# Title

In [1]:
r"""markdown
    TITLE   :
    AUTHOR  :
    PROJECT :
""";

__author__ = ''
__version__ = ''

<span style='font-size:30px;font-weight:650'>
    About
</span>


<br><br>

- - - 
- - - 

<br>

# Prepare

## Imports

In [15]:
## General
# import requests
# from bs4 import BeautifulSoup
# import flask
import pandas as pd

## Custom
from IPython.display import HTML

## Project-Specific

## Functions

In [183]:
def scrape_table(inputurl, columns=[], rename={}, date_col='date', sort=None, save=False, combine_to_master=True):

    # get get from html
    dflist = pd.read_html(inputurl, attrs={"class": ["wikitable","wikitable sortable"]})
    # concatenate and sort
    df = concatenate_tables(*dflist, sort=sort)
    
    # get right columns
    df = df[columns]
    df.rename(columns=rename, inplace=True)

    # add source
    df['source'] = inputurl
    
    df = parse_dates(df)
    
#     if isinstance(save, str):
#         df.to_csv(save, index=False)
#     if combine_to_master:
#         add_to_master_table(df)
    return df


def parse_dates(df):
    return df


def concatenate_tables(*dfs, sort=None):
    cdf = pd.concat(list(dfs))
    if sort is not None:
        cdf.sort_values(sort, inplace=True)
    cdf.reset_index(drop=True, inplace=True)
    return cdf


def add_to_master_table(df):
    try:
        mdf = pd.read_csv('../data/master_table.csv')
    except:
        df.to_csv('../data/master_table.csv', index=False)
    else:
        new_df = concatenate_tables(mdf, df, sort='Date')
        new_df.to_csv('../data/master_table.csv', index=False)
    return df
   

df = scrape_table("https://en.wikipedia.org/wiki/Timeline_of_the_evolutionary_history_of_life",
                  columns=['Date'],
                  rename={'Date': 'date'},
                  date_col='date',
                  save=False
#                   save='../data/wikipedia_evolutionary_history_of_life.csv'
)

df


SyntaxError: invalid syntax (<ipython-input-183-fb9cac7855ed>, line 51)

<br><br>

- - - 
- - - 

<br>

,Date,Event,source
0,4600 Ma,The planet Earth forms from the accretion disc...,https://en.wikipedia.org/wiki/Timeline_of_the_...
1,4500 Ma,"According to the giant impact hypothesis, the ...",https://en.wikipedia.org/wiki/Timeline_of_the_...
2,4404 Ma,First appearance of liquid water on Earth.,https://en.wikipedia.org/wiki/Timeline_of_the_...
3,4280 Ma,Earliest possible appearance of life on Earth....,https://en.wikipedia.org/wiki/Timeline_of_the_...
4,4000 Ma,Formation of a greenstone belt of the Acasta G...,https://en.wikipedia.org/wiki/Timeline_of_the_...
...,...,...,...
142,105 ya (1914),"Martha, last known passenger pigeon, dies",https://en.wikipedia.org/wiki/Timeline_of_the_...
143,83 ya (1936),"The thylacine goes extinct in a Tasmanian zoo,...",https://en.wikipedia.org/wiki/Timeline_of_the_...
144,67 ya (1952),The Caribbean monk seal goes extinct[78],https://en.wikipedia.org/wiki/Timeline_of_the_...
145,11 ya (2008),"The baiji, the Yangtze river dolphin, becomes ...",https://en.wikipedia.org/wiki/Timeline_of_the_...


# Code

In [153]:
# df = scrape_table("https://en.wikipedia.org/wiki/Timeline_of_the_evolutionary_history_of_life", save='../data/wikipedia_evolutionary_history_of_life.csv')

# df = scrape_table('https://en.wikipedia.org/wiki/Timeline_of_natural_history', save='../data/wikipedia_natural_history.csv', combine_to_master=False, sort='Started')
# df

ndf = df[['Period', 'Started', 'source']]
ndf.rename(mapper={'Started': 'started'}, inplace=True)

ndf

,Period,Started,source
0,Tonian,c.1000 Ma,https://en.wikipedia.org/wiki/Timeline_of_natu...
1,Stenian,c.1200 Ma,https://en.wikipedia.org/wiki/Timeline_of_natu...
2,Ectasian,c.1400 Ma,https://en.wikipedia.org/wiki/Timeline_of_natu...
3,Cretaceous,c.145 Ma,https://en.wikipedia.org/wiki/Timeline_of_natu...
4,Calymmian,c.1600 Ma,https://en.wikipedia.org/wiki/Timeline_of_natu...
5,Statherian,c.1800 Ma,https://en.wikipedia.org/wiki/Timeline_of_natu...
6,Quaternary,c.2.58 Ma,https://en.wikipedia.org/wiki/Timeline_of_natu...
7,Jurassic,c.201.3 Ma,https://en.wikipedia.org/wiki/Timeline_of_natu...
8,Orosirian,c.2050 Ma,https://en.wikipedia.org/wiki/Timeline_of_natu...
9,Neogene,c.23.03 Ma,https://en.wikipedia.org/wiki/Timeline_of_natu...


In [143]:
# df = scrape_table('https://en.wikipedia.org/wiki/Timeline_of_the_evolutionary_history_of_life', combine_to_master=False, sort='Date')
# df

,Date,Event,source
0,1 Ma,First coyotes,https://en.wikipedia.org/wiki/Timeline_of_the_...
1,1 bya,The first non-marine eukaryotes move onto land...,https://en.wikipedia.org/wiki/Timeline_of_the_...
2,1.2 Ma,Evolution of Homo antecessor. The last members...,https://en.wikipedia.org/wiki/Timeline_of_the_...
3,1.7 Ma,Extinction of australopithecines,https://en.wikipedia.org/wiki/Timeline_of_the_...
4,10 Ma,"Grasslands and savannas are established, diver...",https://en.wikipedia.org/wiki/Timeline_of_the_...
...,...,...,...
142,90 Ma,Extinction of ichthyosaurs. Earliest snakes an...,https://en.wikipedia.org/wiki/Timeline_of_the_...
143,95 Ma,First crocodilians evolve,https://en.wikipedia.org/wiki/Timeline_of_the_...
144,By 1200 Ma,Meiosis and sexual reproduction are present in...,https://en.wikipedia.org/wiki/Timeline_of_the_...
145,By 1850 Ma,Eukaryotic cells appear. Eukaryotes contain me...,https://en.wikipedia.org/wiki/Timeline_of_the_...


In [166]:
import numpy as np
from astropy.table import Table

data = np.loadtxt('../data/model.txt', skiprows=10)
np.savetxt('../data/model_sub.csv', data[:500, :3], delimiter=',')

<br><br>

- - - 
- - - 

<span style='font-size:40px;font-weight:650'>
    END
</span>